# OpenAI API

In [ ]:
import openai
import os
import re
import sys
import time
sys.path.append(os.getcwd() + '/..')
from scripts import read_transkribus_files

In [ ]:
def get_openai_key(keyfile):
    infile = open(keyfile, "r")
    lines = infile.readlines()
    infile.close()
    return lines[-1].split("=")[-1].strip()

In [ ]:
KEYFILE = "../../openai-quickstart-python/.env"

In [ ]:
openai.api_key = get_openai_key(KEYFILE)

In [ ]:
data_dir = "../../data/Overlijden/x-samples/three-columns-100/page"

texts, metadata, textregions = read_transkribus_files.read_files(data_dir)

In [ ]:
PROMPT_DECEASED_NAME = """Here is a Dutch death certificate. 
Can you extract the name of the deceased person from this document? 
Only give the deceased name as result. 
Other information from the document is not necessary.
""" 

PROMPT_DEATH_DATE = """Here is a Dutch death certificate. 
Can you extract the death date from this document? 
Please present the result in numeric form, with a zero prepending day and month numbers smaller than 10, so for example like: 01-01-1900.
Only give the death date as result. 
Other information from the document is not necessary.
""" 

`openai.ChatCompletion` needs the latest version of the API (v0.27.0+), update with `pip install --upgrade openai`

In [ ]:
responses = {} 
for key in sorted(texts.keys()):
    response = openai.ChatCompletion.create(
       model="gpt-3.5-turbo",
       messages=[
          {"role": "user", "content": PROMPT_DECEASED_NAME + texts[key] },
       ]
    )
    responses[key] = response.choices
    print(responses[key][0]["message"]["content"], key)
    time.sleep(1)

In [ ]:
last_key = "1868-1e-356"
responses = {} 
for key in sorted(texts.keys()):
    if key > last_key:
        response = openai.ChatCompletion.create(
           model="gpt-3.5-turbo",
           messages=[
              {"role": "user", "content": PROMPT_DECEASED_NAME + texts[key] },
           ]
        )
        responses[key] = response.choices
        print(responses[key][0]["message"]["content"], key)
        time.sleep(1)